### Nandini `MDS202335`
### Aritra `MCS202304`

# Importing necessary modules

In [55]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.distributions import Categorical
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import warnings
from prettytable import PrettyTable

warnings.filterwarnings("ignore")

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [19]:
DATA_PATH = "/kaggle/input/sms-spam-collection-dataset"

# Load and preprocess data

In [20]:
df = pd.read_csv(os.path.join(DATA_PATH, "spam.csv"), encoding='latin1')

df = df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
df = df.rename(columns={"v1": "category", "v2": "message"})

df.head(10)

,category,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


# An `END_TOKEN` for `LSTM` and `RNN` model to denote the `end`


 
`"❙"` *Vertical Bold bar* is used to denote the end as it was not in the vocab.


In [21]:
END_TOKEN = '\u2759'

print(END_TOKEN)

❙


# Appending `END_TOKEN` to every message

In [22]:
messages = list(df["message"])

for idx in range(len(messages)):
    messages[idx] += END_TOKEN

In [23]:
messages[80]

"Sorry, I'll call later❙"

### Remove meaningless characters to get rid of `hallucinations`

In [25]:
allowed_chars = [' ', '!', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '❙']

# Build the corpus from the messages
- All the messages are joined to get the corpus
- Unallowed charcters are removed.
- Corpus is truncated for faster training and relevant contexts.

In [27]:
corpus = ' '.join(messages)
corpus = ''.join(char for char in corpus if char in allowed_chars)
corpus = corpus[:80000]
chars = sorted(list(set(corpus))) 

data_size, vocab_size = len(corpus), len(chars) 

print("Corpus has {} characters, {} unique.".format(data_size, vocab_size))

Corpus has 80000 characters, 74 unique.


In [28]:
print(chars)

[' ', '!', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '❙']


# Define the `mapping` and the `inverse mapping` for vocab

In [29]:
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# Convert the stream of `characters` to sequence of `ints`


In [30]:
def get_sequence(input_string):
    """
    Encodes a given string into a list of indices based on the char_to_ix mapping.

    Parameters:
    - input_string (str): The string to be encoded.
    - char_to_ix (dict): A dictionary mapping characters to their respective indices.

    Returns:
    - List[int]: A list of indices corresponding to the characters in the input string.
    """
    sequence = [char_to_ix[ch] for ch in input_string if ch in char_to_ix]
    
    return sequence

In [31]:
corpus = get_sequence(corpus)

In [32]:
# Make the corpus array tensor
data = torch.tensor(corpus).to(device)

# Add an extra dimension to dimension 1
data = torch.unsqueeze(data, dim=1)

## `myRNN` Architecture

The `myRNN` class implements a simple recurrent neural network with configurable layers, hidden size, and dropout:

- **Parameters**:
  - `input_size`: Number of input features in each time step of the sequence.
  - `output_size`: Number of output features in each time step.
  - `hidden_size`: Size of the hidden state vector.
  - `num_layers`: Number of recurrent layers.
  - `do_dropout`: Boolean indicating whether to apply dropout.

- **Components**:
  - **Dropout Layer**: If `do_dropout` is set to `True`, a dropout layer with a dropout rate of 0.5 is applied to the inputs, which helps prevent overfitting.
  - **RNN Layer**: The core of the architecture, an `nn.RNN` layer, processes the input sequence using `num_layers` recurrent layers.
  - **Decoder Layer**: An `nn.Linear` layer maps the final hidden state to the desired `output_size`, producing predictions for each time step.


- **Forward Pass**:
  - The input sequence is first converted to a one-hot encoding with shape `[sequence_length, batch_size, input_size]`.
  - Dropout is applied to the input if enabled.
  - The RNN processes the input, updating the hidden state at each step. The output of the RNN is passed to the decoder to generate predictions.
  - The hidden state is detached to prevent gradient backpropagation through time steps across training batches.

---

## `myLSTM` Architecture

The `myLSTM` class implements a long short-term memory network, which is more complex than the standard RNN due to additional gating mechanisms:

- **Parameters**:
  - Similar to `myRNN`: `input_size`, `output_size`, `hidden_size`, `num_layers`, and `do_dropout`.


- **Components**:
  - **Dropout Layer**: Functions identical to the dropout in `myRNN`.
  - **LSTM Layer**: The `nn.LSTM` layer includes gates that control information flow, helping mitigate issues like vanishing gradients in long sequences.
  - **Decoder Layer**: As with `myRNN`, an `nn.Linear` layer maps the hidden states to the output size for prediction.


- **Forward Pass**:
  - The input sequence is converted to a one-hot encoding.
  - Dropout is applied if enabled.
  - The LSTM processes the input, updating both the hidden state and the cell state at each step. These states are passed to the decoder for predictions.
  - Both the hidden state and cell state are detached to avoid backpropagation through the previous batches.

In [13]:
class myRNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size=512, num_layers=3, do_dropout=False):
        super(myRNN, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.do_dropout = do_dropout
        
        self.dropout = nn.Dropout(0.5)
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
        self.hidden_state = None 
    
    def forward(self, input_seq):
        x = nn.functional.one_hot(input_seq, self.input_size).float()
        if self.do_dropout:
            x = self.dropout(x)
        x, new_hidden_state = self.rnn(x, self.hidden_state)
        output = self.decoder(x)
        self.hidden_state = new_hidden_state.detach() 
        return output
    
    def save_model(self, path):
        torch.save(self.state_dict(), path)
    
    def load_model(self, path):
        try:
            self.load_state_dict(torch.load(path))
        except Exception as err:
            print("Error loading model from file", path)
            print(err)
            print("Initializing model weights to default")
            self.__init__(self.input_size, self.output_size, self.hidden_size, self.num_layers)

class myLSTM(nn.Module):
    def __init__(self, input_size, output_size, hidden_size=512, num_layers=3, do_dropout=False):
        super(myLSTM, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.do_dropout = do_dropout
        
        self.dropout = nn.Dropout(0.5)
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
        self.internal_state = None 
    def forward(self, input_seq):
        x = nn.functional.one_hot(input_seq, self.input_size).float()
        if self.do_dropout:
            x = self.dropout(x)
        x, new_internal_state = self.lstm(x, self.internal_state)
        output = self.decoder(x)
        self.internal_state = (new_internal_state[0].detach(), new_internal_state[1].detach())
        return output
    
    def save_model(self, path):
        torch.save(self.state_dict(), path)
    
    def load_model(self, path):
        try:
            self.load_state_dict(torch.load(path))
        except Exception as err:
            print("Error loading model from file", path)
            print(err)
            print("Initializing model weights to default")
            self.__init__(self.input_size, self.output_size, self.hidden_size, self.num_layers)

# Get Number of Parameters in a Model

- This function calculates the total number of parameters in a given PyTorch model.

- It is useful for comparing model complexity across different architectures.


In [33]:
def get_n_params(model):
    """
    returns the number of parameters of a model
    """
    np=0
    for p in list(model.parameters()):
        np += p.nelement()
    return np

### `train` function to train the LSTM and RNN models

In [34]:
def train(model, epoch, seq_len = 200):
    """
    Train the model on a sequence data for one epoch.

    This function trains the given RNN model for a single epoch using sequence data, 
    computes the average loss, and periodically displays a sample sequence generated by the model.

    Parameters:
    model : nn.Module
        The RNN model to be trained.
    epoch : int
        The current epoch number, used to control logging and sample generation.
    seq_len : int, optional
        The length of the input sequence (default is 200).
    
    Returns:
    float
        The average training loss over all batches for the epoch.
    
    Process:
    - Sets the model to training mode.
    - Defines a cross-entropy loss function.
    - Randomly selects a data pointer within the sequence range.
    - For each batch in the sequence:
      - Extracts the input and target sequences.
      - Computes the model output and the loss.
      - Backpropagates the loss and updates model parameters.
    - Every 10 epochs or during early epochs (1-3), generates and displays a sample output.
      - This sample is generated by feeding the model an initial input and sampling subsequent characters.
    """
    model.train()
    loss_fn = nn.CrossEntropyLoss()
    
    
    test_output_len = seq_len
    
    data_ptr = np.random.randint(seq_len)
    running_loss = 0
    n = 0;
    
    if epoch % 10 == 0 or epoch == 1 or epoch == 2 or epoch == 3:
        print("\nStart of Epoch: {0}".format(epoch))
        
    while True:
        input_seq = data[data_ptr : data_ptr+seq_len]
        target_seq = data[data_ptr+1 : data_ptr+seq_len+1]
        input_seq.to(device)
        target_seq.to(device)
        
        optimizer.zero_grad()
        output = model(input_seq)
        loss = loss_fn(torch.squeeze(output), torch.squeeze(target_seq))
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        data_ptr += seq_len
        if data_ptr + seq_len + 1 > data_size:
            break
        
        n = n+1
            
    if epoch % 10 == 0 or epoch == 1 or epoch == 2 or epoch == 3:
        model.eval()
        data_ptr = 0

        rand_index = np.random.randint(data_size-1)
        input_seq = data[rand_index : rand_index+1]

        
        test_output = ""
        while True:
            output = model(input_seq)

            output = F.softmax(torch.squeeze(output), dim=0)
            dist = Categorical(output)
            index = dist.sample().item()
            

            test_output += ix_to_char[index]

            input_seq[0][0] = index
            data_ptr += 1

            if data_ptr > test_output_len:
                break
        print("----------")
        print("TRAIN Sample")
        print(test_output)
        print("----------")
        print("End of Epoch: {0} \t Loss: {1:.8f}".format(epoch, running_loss / n))
    
    return running_loss / n

### `test` function to evaluate the performance of message completion

In [ ]:
def test(model, x, output_len=50, T=0): 
    """
    Generate a sequence of characters using the provided model based on an initial input string.

    This function takes an initial input string, encodes it as indices, and uses the 
    specified RNN model to generate a sequence of characters of a given length. 
    The generation can be influenced by the temperature parameter (T) to control randomness.

    Parameters:
    model : nn.Module
        The trained RNN model to be used for character generation.
    x : str
        The initial input string from which to start generating the output sequence.
    output_len : int, optional
        The maximum length of the generated output sequence (default is 50).
    T : float, optional
        The temperature parameter that influences the randomness of the predictions. 
        A value of 0 means that the model will always select the most probable character 
        (greedy approach). A value other than 1 will sample from the output probability distribution.

    Returns:
    None

    Process:
    - The model is set to evaluation mode.
    - The input string is converted to a tensor of character indices.
    - A subset of the input sequence is selected to be used for generation.
    - The model generates characters one at a time until the specified output length 
      is reached or an end token is encountered.
    - The generated characters are accumulated into a result string and printed along 
      with the initial input context.
    """
    model.eval()
    
    data_ptr = 0
    hidden_state = None    
    
    
    input_seq = torch.tensor([char_to_ix[char] for char in x]).to(device)
    input_seq = torch.unsqueeze(input_seq, dim=1)
    y = 30
    y = min(y, len(input_seq) // 2)
    x = 0
    input_seq = input_seq[x:x+y]
    test_input = ''.join(ix_to_char[int(ix)] for ix in input_seq)

    output = model(input_seq)
    
    input_seq = data[x+y:x+y+1]
    
    test_output = ""
    
    while True:
        output = model(input_seq)
        
        output = F.softmax(torch.squeeze(output), dim=0)
        dist = Categorical(output)
        index = dist.sample().item()
        
        most_probable_index = torch.argmax(output).item()
        
        # If temperature T is set to 0, we take the most probable one always.
        if T == 0:
            index = most_probable_index
        test_output += ix_to_char[index]
        
        if ix_to_char[index] == END_TOKEN:
            break
        
        input_seq[0][0] = index
        data_ptr += 1
        
        if data_ptr  > output_len:
            break

    print(f"{model.__class__.__name__} completed:")
    print(f"context: {test_input}")
    print(f"output: {test_output}")

# Training the LSTM model

In [ ]:
hidden_size = 512
num_layers = 3       
lr = 0.002          

model_lstm = myLSTM(vocab_size, vocab_size, hidden_size, num_layers).to(device)
optimizer = torch.optim.Adam(model_lstm.parameters(), lr=lr)

best_model_lstm = myLSTM(vocab_size, vocab_size, hidden_size, num_layers).to(device)
best_lstm_loss = 10000
 
for epoch in tqdm(range(1, 101)):
    epoch_loss = train(model_lstm, epoch, 50)
    if epoch_loss < best_lstm_loss:
        best_lstm_loss = epoch_loss
        best_model_lstm.load_state_dict(model_lstm.state_dict())

  0%|          | 0/100 [00:00<?, ?it/s]


Start of Epoch: 1
----------
TRAIN Sample
lock that intr u tear of doing tho end to MANESS! I
----------
End of Epoch: 1 	 Loss: 0.28820632

Start of Epoch: 2
----------
TRAIN Sample
ok, I beilge in the pls on my somect.❙ I lave m che
----------
End of Epoch: 2 	 Loss: 0.22217210

Start of Epoch: 3
----------
TRAIN Sample
ng about the mistumf ot the u'll then i go soworrow
----------
End of Epoch: 3 	 Loss: 0.13288940

Start of Epoch: 10
----------
TRAIN Sample
re.❙ As per u bit of people that dont kear and wank
----------
End of Epoch: 10 	 Loss: 0.01264650

Start of Epoch: 20
----------
TRAIN Sample
er already?❙ Yup... Ok lar... Joking wif u oni...❙ 
----------
End of Epoch: 20 	 Loss: 0.02349879

Start of Epoch: 30
----------
TRAIN Sample
that be a win FA Cup final tkts 2 a wAly comp to wi
----------
End of Epoch: 30 	 Loss: 0.05197851

Start of Epoch: 40
----------
TRAIN Sample
rconter! Nit me know so nice is so itker ereding❙ Y
----------
End of Epoch: 40 	 Loss: 0.14801698

Star

In [349]:
def complete_message_using_lstm(index, T=0):
    """
    A function that takes the first half of the message to return the remaining part.
    """
    print("original message:")
    print(messages[index])
    test(best_model_lstm, x=messages[index], T=0)

# Performance of the `LSTM` on the unseen data

In [357]:
complete_message_using_lstm(4675)

original message:
Probably money worries. Things are coming due and i have several outstanding invoices for work i did two and three months ago.❙
LSTM completed:
context: Probably money worries. Things
output: for making me ? I been egilly selected 2 receive 10


In [297]:
complete_message_using_lstm(3786)

original message:
Aight, I'll hit you up when I get some cash❙
LSTM completed:
context: Aight, I'll hit you up when I 
output: ervice representative on 0800 169 6031 between 10am


In [298]:
complete_message_using_lstm(5663)

original message:
I am waiting machan. Call me once you free.❙
LSTM completed:
context: I am waiting machan. Call me o
output: ce ic Smile Qursh on 5800 169 6031 between 10am-9pm


In [328]:
complete_message_using_lstm(1239)

original message:
Hope you are having a great new semester. Do wish you the very best. You are made for greatness.❙
LSTM completed:
context: Hope you are having a great ne
output:  You are a winner U have been specially selected 2 


# Performance of the `LSTM` on the previously seen data.

In [305]:
complete_message_using_lstm(0)

original message:
Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...❙
LSTM completed:
context: Go until jurong point, crazy..
output: Alright you liked it, since i was doing the best i 


In [306]:
complete_message_using_lstm(10)

original message:
I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.❙
LSTM completed:
context: I'm gonna be home soon and i d
output: es not the money so carlos can make the call❙


In [321]:
complete_message_using_lstm(1233)

original message:
Lol ok. I'll snatch her purse too.❙
LSTM completed:
context: Lol ok. I'll snatch her purse 
output: How?❙


# Training the RNN model

In [37]:
hidden_size = 1024 
num_layers = 3       
lr = 0.002          

model_rnn = myRNN(vocab_size, vocab_size, hidden_size, num_layers).to(device)
optimizer = torch.optim.Adam(model_rnn.parameters(), lr=lr)

best_model_rnn = myRNN(vocab_size, vocab_size, hidden_size, num_layers).to(device)
best_rnn_loss = 10000
 
for epoch in tqdm(range(1, 101)):
    epoch_loss = train(model_rnn, epoch, 50)
    if epoch_loss < best_rnn_loss:
        best_rnn_loss = epoch_loss
        best_model_rnn.load_state_dict(model_rnn.state_dict())

  0%|          | 0/100 [00:00<?, ?it/s]


Start of Epoch: 1
----------
TRAIN Sample
G np c agor.rLgaa osTei  esg E  dan a-d.i possoos i
----------
End of Epoch: 1 	 Loss: 3.84326125

Start of Epoch: 2
----------
TRAIN Sample
imuiaEuaz ppigaozcntbir .tW ialf tgupwno? uu ipL au
----------
End of Epoch: 2 	 Loss: 3.81743740

Start of Epoch: 3
----------
TRAIN Sample
leowdddtoogi odaoiooea  oetddnociimg terrna gsrdit 
----------
End of Epoch: 3 	 Loss: 3.81418707

Start of Epoch: 10
----------
TRAIN Sample
0 c  aicntpo rds rn  p gaRswiaioutt gr tdoceedn iud
----------
End of Epoch: 10 	 Loss: 3.81446393

Start of Epoch: 20
----------
TRAIN Sample
 s0pr  ooer0rns osigwiidr  onRi ggidrcntaBdla ts nL
----------
End of Epoch: 20 	 Loss: 3.81933014

Start of Epoch: 30
----------
TRAIN Sample
idzoRmurziisncgatus ddcraL emtdRi uncgcnir eae  zii
----------
End of Epoch: 30 	 Loss: 3.81765289

Start of Epoch: 40
----------
TRAIN Sample
o .n e gugeptp din nidni B r urruam  roir0ntrcdc .m
----------
End of Epoch: 40 	 Loss: 3.81746325

Star

In [39]:
def complete_message_using_rnn(index):
    """
    A function that takes the first half of the message to return the remaining part.
    """
    print("original message:")
    print(messages[index])
    test(best_model_rnn, x=messages[index], T=1)

# Performance of the `RNN` on the unseen data

In [41]:
complete_message_using_rnn(4399)

original message:
Juz go google n search 4 qet...❙
myRNN completed:
context: Juz go google n 
output:  t.iadnsaL❙


In [42]:
complete_message_using_rnn(2987)

original message:
Do you still have the grinder?❙
myRNN completed:
context: Do you still ha
output: scrr rrpi dnoecnncobiiot paooiss❙


In [43]:
complete_message_using_rnn(4553)

original message:
Try to do something dear. You read something for exams❙
myRNN completed:
context: Try to do something dear. Y
output: ?eBiitiodrdsn int senongyaiiw aagoinnLmrfEunogngcd 


In [48]:
complete_message_using_rnn(3019)

original message:
I thank you so much for all you do with selflessness. I love you plenty.❙
myRNN completed:
context: I thank you so much for all yo
output: nidni niantodp.gnhtno  od .np ssdi np ndcaw tdp.sza


# Performance of the `RNN` on the previously seen data.

In [40]:
complete_message_using_rnn(0)

original message:
Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...❙
myRNN completed:
context: Go until jurong point, crazy..
output: iIigs  .ae  snnpHnnomsonkt GGcgdttnL sniinsghddoi g


In [49]:
complete_message_using_rnn(13)

original message:
I've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times.❙
myRNN completed:
context: I've been searching for the ri
output: ❙


In [51]:
complete_message_using_rnn(64)

original message:
Ok lar i double check wif da hair dresser already he said wun cut v short. He said will cut until i look nice.❙
myRNN completed:
context: Ok lar i double check wif da h
output: iesrdstatnndr MagLrutnnitaintntsgdn oemn uwunscinci


In [56]:
rnn_params = get_n_params(best_model_rnn)
lstm_params = get_n_params(best_model_lstm)

table = PrettyTable()
table.field_names = ["Model", "Number of Parameters"]
table.add_row(["RNN Model", rnn_params])
table.add_row(["LSTM Model", lstm_params])

print(table)

+------------+----------------------+
|   Model    | Number of Parameters |
+------------+----------------------+
| RNN Model  |       5400650        |
| LSTM Model |       5444682        |
+------------+----------------------+


## Comparison between `LSTM` and `RNN` model

In [ ]:
def compare_completion(index):
    """
    A function that takes the first half of the message and compares rnn and lstm output for the remaining part.
    """
    print("original message:")
    print(messages[index])
    print("\n")
    test(best_model_rnn, x=messages[index], T=1)
    print("\n")
    test(best_model_lstm, x=messages[index], T=0)

# Comparison on unseen data

In [65]:
compare_completion(4000)

original message:
He's just gonna worry for nothing. And he won't give you money its no use.❙


myRNN completed:
context: He's just gonna worry for noth
output: oLlnu inisntnnagatonr.nmgo .L  z.tai  cu .iaitnnrs.


myLSTM completed:
context: He's just gonna worry for noth
output:  Txll be show you sorry your reply...❙


In [66]:
compare_completion(4567)

original message:
hiya hows it going in sunny africa? hope u r avin a good time. give that big old silver back a big kiss from me.❙


myRNN completed:
context: hiya hows it going in sunny af
output: d!caatogtty rtIwadh.  gtnBncana mnns..nenid amomtgn


myLSTM completed:
context: hiya hows it going in sunny af
output:  that i have hairdressers appointment at four so ne


In [69]:
compare_completion(3097)

original message:
This is all just creepy and crazy to me.❙


myRNN completed:
context: This is all just cre
output: oisn d oaudo iensrouruggn1rte.0ie iad w Rsrdtpiip i


myLSTM completed:
context: This is all just cre
output: sht? He said will cut until i look nice.❙


In [79]:
compare_completion(4880)

original message:
When/where do I pick you up❙


myRNN completed:
context: When/where do 
output:   . aa tan zgroyH  atag RegBsd tsr.o prnepoea zemei


myLSTM completed:
context: When/where do 
output: t seemed so happy about the cave. I'm sorry i did. 


In [81]:
compare_completion(4006)

original message:
I'm reaching home in 5 min.❙


myRNN completed:
context: I'm reaching h
output: o  ndLddsGnpco -ta  caLtoogppncnnna oanato dahttiio


myLSTM completed:
context: I'm reaching h
output: w dying an egg ? Did you makeoa tea? Are you eating


# Comparison on seen data

In [72]:
compare_completion(40)

original message:
Pls go ahead with watts. I just wanted to be sure. Do have a great weekend. Abiola❙


myRNN completed:
context: Pls go ahead with watts. I jus
output: oari s nidzi anGog s❙


myLSTM completed:
context: Pls go ahead with watts. I jus
output:  wanted to be sure. Do have a great weekend. Abiola


In [83]:
compare_completion(0)

original message:
Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...❙


myRNN completed:
context: Go until jurong point, crazy..
output: onrtRaSriarwsr.i.csdccssirp rgs.htsBduBsoa-e pao.ao


myLSTM completed:
context: Go until jurong point, crazy..
output: Available only in bosty no hand did? Quick have a c


In [84]:
compare_completion(10)

original message:
I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.❙


myRNN completed:
context: I'm gonna be home soon and i d
output: Lia hhrpamd7htiis .sg reggtrnnoMtngdirngndot   ezpe


myLSTM completed:
context: I'm gonna be home soon and i d
output: er that i hours on your man well check all thk i co


In [85]:
compare_completion(1233)

original message:
Lol ok. I'll snatch her purse too.❙


myRNN completed:
context: Lol ok. I'll snat
output: hn L tzi antcioopiz.hdsiituuncp nLngniitBn ets os s


myLSTM completed:
context: Lol ok. I'll snat
output:  a 1500 prize Jackpot! Txt ur national team to 8707


In [86]:
compare_completion(1239)

original message:
Hope you are having a great new semester. Do wish you the very best. You are made for greatness.❙


myRNN completed:
context: Hope you are having a great ne
output: oo icuigcpiiR L.tinctigssLidn odOttrddgotp ndSismut


myLSTM completed:
context: Hope you are having a great ne
output:  this saymur star sign, e. go ahead with watts. I j


In [87]:
compare_completion(50)

original message:
What you thinked about me. First time you saw me in class.❙


myRNN completed:
context: What you thinked about me. Fi
output:  sonosoaDnri et   odsegstew a totpfhoG❙


myLSTM completed:
context: What you thinked about me. Fi
output: st time you saw me in class.❙


# Conclusion
We can see the `LSTM` works far better than `RNN` when use as a `generator`.

- From the output on seen data, `LSTM` din't memorize the sequence, yet yielded completions of moderately high quality, even the sentences were meaningful for `LSTM`. Such a high-quality completion is shown below.
 ```
 original message:
What you thinked about me. First time you saw me in class.❙

---
myRNN completed:
context: What you thinked about me. Fi
output:  sonosoaDnri et   odsegstew a totpfhoG❙
---
myLSTM completed:
context: What you thinked about me. Fi
output: st time you saw me in class.❙
```
- In contrast, `RNN` generated gibberish texts all the time.

So, we can infer that `long term memory` of `LSTM` really made a difference.